# dbachecks PR check on Beard NUC - MultipleInstances

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local source control and ensure that we are in the PR branch

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
    git status
    

On branch pr/tboggiano/759
Your branch is ahead of 'tboggiano/ports' by 12 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   checks/Server.Tests.ps1
	modified:   internal/configurations/DbcCheckDescriptions.json
	modified:   internal/configurations/configuration.ps1

no changes added to commit (use "git add" and/or "git commit -a")


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1 -Verbose
$null= Reset-DbcConfig 

VERBOSE: Loading module from path 'D:\OneDrive\Documents\GitHub\dbachecks\dbachecks.psd1'.
VERBOSE: Loading 'FormatsToProcess' from path 'D:\OneDrive\Documents\GitHub\dbachecks\xml\dbachecks.Format.ps1xml'.
VERBOSE: Loading module from path 'D:\OneDrive\Documents\GitHub\dbachecks\dbachecks.psm1'.
VERBOSE: Exporting function 'Import-ModuleFile'.
VERBOSE: Exporting function 'Get-CheckFile'.
VERBOSE: Exporting function 'Get-CheckInformation'.
VERBOSE: Exporting function 'Get-CheckRepo'.
VERBOSE: Exporting function 'Get-ComputerName'.
VERBOSE: Exporting function 'Get-DatabaseDetail'.
VERBOSE: Exporting function 'Get-Instance'.
VERBOSE: Exporting function 'Get-Version'.
VERBOSE: Exporting function 'Invoke-ConfigurationScript'.
VERBOSE: Exporting function 'New-Json'.
VERBOSE: Exporting function 'Select-DefaultView'.
VERBOSE: Exporting function 'Set-DatabaseForIntegrationTesting'.
VERBOSE: Exporting function 'Clear-DbcPowerBiDataSource'.
VERBOSE: Exporting function 'Export-DbcConfig'.
VERBOSE

Excellent, that worked. Now to run some checks against a single instance  

I don't care if the test pass or fail, I am only interested in any 

> Describe block has Error

or

> Error in Script

type failures

Multiple Instances at parameter

# Make sure that the check and config have been written to the json file

In [3]:
Get-DbcCheck NonStandardPort | fl


Group       : Server
Type        : ComputerName
UniqueTag   : NonStandardPort
AllTags     : NonStandardPort, Medium, CIS, Server
Config      : app.computername skip.security.nonstandardport 
Description : Tests to see if the SQL Server Instances are NOT running on the default port of 1433.




In [4]:
Get-DbcConfig skip.security.NonStandardPort | fl


Name        : skip.security.nonstandardport
Value       : True
Description : Skips the check for whether SQL Server should be configured with a non standard port




In [5]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Invoke-DbcCheck -ComputerName $Sqlinstances -Check NonStandardPort

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1' with Tags NonStandardPort

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1

  Describing Non Standard Port

    Context Checking SQL Server ports on SQL2005Ser2003
      [!] No SQL Server Instances should be configured with port 1433 on SQL2005Ser2003, is skipped 0ms

  Describing Non Standard Port

    Context Checking SQL Server ports on SQL2016N2
      [!] No SQL Server Instances should be configured with port 1433 on SQL2016N2, is skipped 0ms

  Describing Non Standard Port

    Context Checking SQL Server ports on SQL2012Ser08AG1
      [!] No SQL Server Instances should be configured with port 1433 on SQL2012Ser08AG1, is skipped 0ms

  Describing Non Standard Port

    Context Checking SQL Server ports

Set the config and run again

In [17]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Set-DbcConfig -Name skip.security.nonstandardport -Value $false
Invoke-DbcCheck -ComputerName SQL2017N5 -Check NonStandardPort


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1' with Tags NonStandardPort

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Server.Tests.ps1

  Describing Non Standard Port

    Context Checking SQL Server ports on SQL2017N5
      [+] No SQL Server Instances should be configured with port 1433 on SQL2017N5 3ms
Tests completed in 281ms
Tests Passed: 1, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 
Name                          Value Description
----                          ----- -----------
skip.security.nonstandardport False Skips the check for whether SQL Server should be configured wi…



In [15]:
Get-DbcConfig -Name global.notcontactable 


Name                  Value Description
----                  ----- -----------
global.notcontactable {}    This is used within the checks to avoid trying to contact none-respons…



In [9]:
@(Get-DbaLogin -SQLInstance sql2019n5 -Type SQL | Where-Object { $_.PasswordPolicyEnforced -eq $false -and $_.IsDisabled -eq $false}).Count

1


In [10]:
@(Get-DbaLogin -SqlInstance sql2019n5 -Login sa).Count

1
